# import required libraries


In [1]:
# Ensure OpenCV is installed

import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


In [2]:
# Ensure matplotlib is installed
#!pip install matplotlib

import matplotlib.pyplot as plt

In [3]:
# Define directories for training and test data
train_dir = r"C:\Users\ganga\Documents\IISc Coursework\ML4CPS\Project1\Project 1 Data\Project 1 Data\Train_Data"
test_dir = r"C:\Users\ganga\Documents\IISc Coursework\ML4CPS\Project1\Project 1 Data\Project 1 Data\Test_Data"

In [4]:
# Step 2: Load and Preprocess Training Data
IMG_SIZ = (400, 300)
#IMG_SIZE = (400, 300)
ALLOWED_EXTENSIONS = {'.jpg', '.jpeg', '.png'}

In [5]:
# Function to load training data
def load_data(train_dir):
    images = []
    labels = []
    label_map = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'S': 5}

    for folder in os.listdir(train_dir):
        folder_path = os.path.join(train_dir, folder)
        if os.path.isdir(folder_path) and folder in label_map:
            for img_name in os.listdir(folder_path):
                if not any(img_name.lower().endswith(ext) for ext in ALLOWED_EXTENSIONS):
                    continue
                img_path = os.path.join(folder_path, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, IMG_SIZ)
                    images.append(img)
                    labels.append(label_map[folder])
    return np.array(images), np.array(labels)



In [16]:
# Load training data
if os.path.exists(train_dir):
    images, labels = load_data(train_dir)
    print(f"Loaded {len(images)} images.")
    print(f"Shape of images array: {images.shape}")
    print(f"Shape of labels array: {labels.shape}")

# Step 3: Prepare Data for Training
labels = labels - 1  # Adjust labels to be 0-based

Loaded 2516 images.
Shape of images array: (2516, 300, 400, 3)
Shape of labels array: (2516,)


In [8]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [9]:
# Advanced Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=30,  # Reduce rotation range
    width_shift_range=0.2,  # Reduce shift range
    height_shift_range=0.2,  # Reduce shift range
    shear_range=0.2,  # Reduce shear range
    zoom_range=0.2,  # Reduce zoom
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1./255
)

In [10]:

val_datagen = ImageDataGenerator(rescale=1./255)



In [12]:
#Add augmented data to training set
augmented_images = []
augmented_labels = []



for X_batch, y_batch in datagen.flow(images, labels, batch_size=32):
    augmented_images.append(X_batch)
    augmented_labels.append(y_batch)
    if len(augmented_images) * len(X_batch) >= 5000:
        break
augmented_images = np.concatenate(augmented_images)
augmented_labels = np.concatenate(augmented_labels)
X_train = np.concatenate([X_train, augmented_images])
y_train = np.concatenate([y_train, augmented_labels])
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")

MemoryError: Unable to allocate 43.9 MiB for an array with shape (32, 300, 400, 3) and data type float32

In [20]:
# Define model architecture
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),  # Reduce neurons in dense layer
    Dropout(0.3),  # Reduce dropout rate to retain more information
    Dense(256, activation='relu'),  # Additional dense layer
    Dropout(0.3),
    Dense(5, activation='softmax')  # Assuming 5 classes
])

In [21]:

# Define model architecture
model = Sequential([
    base_model,
    Flatten(),
    
    pout(0.3),
    Dense(5, activation='softmax')  # Assuming 5 classes
])

In [22]:

# Compile the model with AdamW optimizer
model.compile(optimizer=AdamW(learning_rate=0.00005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [23]:
# Step 5: Training the Model
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00005)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [61]:
# Compute class weights (optional: you can experiment with and without this)
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = {i: weight for i, weight in enumerate(class_weights)}

batch_size = 10  # Use a smaller batch size

In [25]:
# Train the model with data augmentation, class weights, and callbacks
model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
          validation_data=val_datagen.flow(X_val, y_val),
          epochs=50,  # Increase epochs for better convergence
          class_weight=class_weights,  # Experiment with and without this
          callbacks=[reduce_lr, early_stopping])  # Add early stopping to prevent overfitting


Epoch 1/40


c:\Users\ganga\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


63/63 ━━━━━━━━━━━━━━━━━━━━ 78s 753ms/step - accuracy: 0.2456 - loss: 1.9624 - val_accuracy: 0.0734 - val_loss: 1.7107 - learning_rate: 5.0000e-05
Epoch 2/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 51s 803ms/step - accuracy: 0.2918 - loss: 1.6920 - val_accuracy: 0.3710 - val_loss: 1.5876 - learning_rate: 5.0000e-05
Epoch 3/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 49s 764ms/step - accuracy: 0.3083 - loss: 1.6187 - val_accuracy: 0.1528 - val_loss: 1.6177 - learning_rate: 5.0000e-05
Epoch 4/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 49s 765ms/step - accuracy: 0.3551 - loss: 1.5662 - val_accuracy: 0.2183 - val_loss: 1.6016 - learning_rate: 5.0000e-05
Epoch 5/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 51s 800ms/step - accuracy: 0.3870 - loss: 1.4566 - val_accuracy: 0.1290 - val_loss: 1.6935 - learning_rate: 5.0000e-05
Epoch 6/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 52s 813ms/step - accuracy: 0.3852 - loss: 1.5043 - val_accuracy: 0.1528 - val_loss: 1.7013 - learning_rate: 5.0000e-05
Epoch 7/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 49s 768ms/step - accuracy: 0.39

In [60]:
# Evaluate the model on the validation set and find accuracy
loss, accuracy = model.evaluate(val_datagen(X_val, y_val))
print(f"Validation accuracy: {accuracy:.2f}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 175ms/step - accuracy: 0.2949 - loss: 1.6977
Validation accuracy: 0.28


In [26]:

# Step 6: Load and Preprocess Test Data with Missing Image Handling
def load_test_data(test_dir):
    test_images = []
    test_ids = []
    missing_images = []
    for img_name in os.listdir(test_dir):
        img_path = os.path.join(test_dir, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (128, 128))
            test_images.append(img)
            test_ids.append(img_name.split('.')[0])
        else:
            print(f"Warning: Could not load image {img_name}")
            missing_images.append(img_name)

    print(f"Loaded {len(test_images)} images.")
    print(f"Missing images: {missing_images}")

    return np.array(test_images), test_ids, missing_images




In [27]:
# Load test data
test_images, test_ids, missing_images = load_test_data(test_dir)

# Normalize test data
test_images = test_images / 255.0

Loaded 478 images.
Missing images: ['.DS_Store']


In [28]:
# Step 7: Make Predictions
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1) + 1  # Adjust to match original labels

15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 231ms/step


In [29]:
# Ensure predicted_classes has 479 entries
if len(predicted_classes) < 479:
    print(f"Expected 479 images, but loaded {len(test_ids)}. Adding placeholder rows for missing images.")

    # Initialize predicted_classes if not already done
    predicted_classes = np.array(predicted_classes)

    # Add placeholder predictions (e.g., class 1) to match the expected number of rows
    for missing_img in missing_images:
        test_ids.append(missing_img.split('.')[0])
        predicted_classes = np.append(predicted_classes, [1])  # Default prediction as class 1

Expected 479 images, but loaded 478. Adding placeholder rows for missing images.


In [30]:
# Step 8: Create Submission File
submission = pd.DataFrame({
    'ID': test_ids,
    'Predictions': predicted_classes
})

In [33]:
# Save submission variable as a .csv file in the current working directory
submission.to_csv('submission.csv', index=False)
print("Submission file created successfully.")


Submission file created successfully.
